In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210716-1318-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [2]:
mt_trios = hl.read_matrix_table(
        f'{lustre_dir}/variant_qc/MegaWES_trios_adj.mt')

In [3]:
mt_trios.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'id': str
    'proband': struct {
        s: str, 
        cohort: str
    }
    'father': struct {
        s: str, 
        cohort: str
    }
    'mother': struct {
        s: str, 
        cohort: str
    }
    'is_female': bool
    'fam_id': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: i

In [9]:
mt_trios.proband.show()

+-------------------+-------------------+----------------+
| id                | proband.s         | proband.cohort |
+-------------------+-------------------+----------------+
| str               | str               | str            |
+-------------------+-------------------+----------------+
| "EGAN00001351469" | "EGAN00001351469" | "DDD"          |
| "EGAN00001315854" | "EGAN00001315854" | "DDD"          |
| "EGAN00001351763" | "EGAN00001351763" | "DDD"          |
| "EGAN00001292231" | "EGAN00001292231" | "DDD"          |
| "EGAN00001352604" | "EGAN00001352604" | "DDD"          |
| "EGAN00001305535" | "EGAN00001305535" | "DDD"          |
| "EGAN00001303416" | "EGAN00001303416" | "DDD"          |
| "EGAN00001304382" | "EGAN00001304382" | "DDD"          |
| "EGAN00001300962" | "EGAN00001300962" | "DDD"          |
| "EGAN00001313595" | "EGAN00001313595" | "DDD"          |
+-------------------+-------------------+----------------+
showing top 10 rows

In [15]:
accessions=hl.import_table(f'{lustre_dir}/kaitlin_trios/forPavlos_100trios_EGA_accessions.txt',no_header=False).key_by('s')   
samples=set(accessions.s.collect())
set_samples=hl.literal(samples)
mt_100_trios=mt_trios.filter_cols(
    (set_samples.contains(mt_trios.proband.s)) & 
    (set_samples.contains(mt_trios.father.s)) & 
    (set_samples.contains( mt_trios.mother.s)))


2021-07-16 13:34:16 Hail: INFO: Reading table without type imputation
  Loading field 's' as type str (not specified)


In [3]:
accessions=hl.import_table(f'{lustre_dir}/kaitlin_trios/forPavlos_100trios_EGA_accessions.txt',no_header=False).key_by('s')   
mt_trios = hl.read_matrix_table(
        f'{lustre_dir}/variant_qc/MegaWES_trios_adj.mt')
samples=accessions.s.collect()

mt_100_trios=mt_trios.filter_cols(samples.contains(mt_trios['s']))

2021-07-16 13:08:10 Hail: INFO: Reading table without type imputation
  Loading field 's' as type str (not specified)


KeyboardInterrupt: 

In [ ]:
mt_trios.count()

In [ ]:
mt_100_trios.count()